<a href="https://colab.research.google.com/github/Ava-creates/adaboost/blob/main/adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
data = load_breast_cancer()

X=data["data"]
y=data["target"]
y.reshape((X.shape[0],1))

print(X)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [ ]:
print(X.shape)
print(y.shape)
print(data.feature_names)


(569, 30)
(569,)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn import preprocessing
scaler_train = preprocessing.StandardScaler().fit(X_train)
scaler_test = preprocessing.StandardScaler().fit(X_test)
scale=preprocessing.StandardScaler().fit(X)
X_train_scaled = scaler_train.transform(X_train)
X_test_scaled= scaler_test.transform(X_test)
X_scaled=scale.transform(X)

In [ ]:
class Classifier:
  
  def __init__(self):
    self.threshold=0
    self.sign= 1
    self.feature= 0
    self.alpha = 0

  def prediction(self, X):
    y_predict= np.ones(X.shape[0])
    X_j= X[:, self.feature]
    

    if(self.sign ==1):
      y_predict[X_j<self.threshold]=-1

    else:

      y_predict[X_j<self.threshold]=1

    return y_predict

  def cal_alpha(self, min_error):
     
    self.alpha = 0.5 * np.log((1.0 - min_error) / (min_error))



In [ ]:
class ensembeled_adaboost:
  def __init__ (self, no_classifiers):
    self.no_classifiers=no_classifiers
    self.classifier_list= []

  def find( self, classifiers, X, y, w):
    a=classifiers
    
    features=X.shape[1]
    min_error=1
    feature=0
    
    for i in range(0, features):
      X_j = X[:, i]
      t = np.unique(X_j)
      row= X.shape[0]
      for t_ in t:
        
        y_estimate = np.ones(X.shape[0])
        sign=1
        
        y_estimate[X_j < t_]= -1

        error= sum(w[y_estimate!=y])/row

        if(error>.5):
          sign=-1
          error=1-error

        if(error<min_error):
          min_error= error
          a.feature=i
          a.sign=sign
          a.threshold= t_

      a.cal_alpha(min_error)

      

  def ensemble(self, X, y):
      row= X.shape[0]
      w = np.full(row, (1 / row))
      for i in range(0, self.no_classifiers):

        classifiers  = Classifier()

        self.find(classifiers, X, y, w)

        y_predict = classifiers.prediction(X)

        w = w*np.exp(-classifiers.alpha * y * y_predict)/ np.sum(w)
         
        self.classifier_list.append(classifiers)

  def predict(self, X):
        
        a= np.zeros(X.shape[0])

        a=a.reshape(X.shape[0],1)


        for classifier in self.classifier_list:
          #print((classifier.alpha * classifier.prediction(X)).shape)
          #print(a.shape)
          a = a+ classifier.alpha * (classifier.prediction(X).reshape(X.shape[0], 1))

        y_pred = a
        y_pred = np.sign(y_pred)

        return y_pred


       






In [ ]:
from sklearn.metrics import accuracy_score
c = ensembeled_adaboost(10)
c.ensemble(X_train, y_train)
y_predicted=c.predict(X_test)

print("Test Accuracy: {}".format(accuracy_score(y_test, y_predicted)))



Test Accuracy: 0.6382978723404256


In [ ]:
from numpy import genfromtxt

my_data = genfromtxt('/content/drive/MyDrive/Colab Notebooks/spambase.csv', delimiter=',')
print(my_data.shape)



label=my_data[:, -1]
data= my_data[: , :-1]
print(label)
print(data)
label= np.where(label ==0, -1, 1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)

from sklearn import preprocessing
scaler_train = preprocessing.StandardScaler().fit(X_train)
scaler_test = preprocessing.StandardScaler().fit(X_test)
scale=preprocessing.StandardScaler().fit(X)
X_train_scaled = scaler_train.transform(X_train)
X_test_scaled= scaler_test.transform(X_test)
X_scaled=scale.transform(X)

(4601, 58)
[1. 1. 1. ... 0. 0. 0.]
[[0.000e+00 6.400e-01 6.400e-01 ... 3.756e+00 6.100e+01 2.780e+02]
 [2.100e-01 2.800e-01 5.000e-01 ... 5.114e+00 1.010e+02 1.028e+03]
 [6.000e-02 0.000e+00 7.100e-01 ... 9.821e+00 4.850e+02 2.259e+03]
 ...
 [3.000e-01 0.000e+00 3.000e-01 ... 1.404e+00 6.000e+00 1.180e+02]
 [9.600e-01 0.000e+00 0.000e+00 ... 1.147e+00 5.000e+00 7.800e+01]
 [0.000e+00 0.000e+00 6.500e-01 ... 1.250e+00 5.000e+00 4.000e+01]]


In [ ]:
from sklearn.metrics import accuracy_score
c = ensembeled_adaboost(50)
c.ensemble(X_train, y_train)
y_predicted=c.predict(X_test)

print("Test Accuracy: {}".format(accuracy_score(y_test, y_predicted)*100))

Test Accuracy: 79.78933508887425
